In [ ]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import os
from dotenv import load_dotenv
import json

In [ ]:
import sys
sys.path.append('./lib')

from pool_v3 import PoolV3
from abi_holder import ABIHolder

In [ ]:
def get_gas_price(mode="market"):
    current_price = w3.eth.gas_price / 1e9
    
    if mode == "low":
        return round(current_price * 0.9)
    elif mode == "high":
        return round(current_price * 1.1)
    elif mode == "asap":
        return round(current_price * 2)
    else:
        return round(current_price)

In [ ]:
load_dotenv()

abi_holder = ABIHolder()

w3 = Web3(Web3.HTTPProvider(os.environ.get("RPC_POLYGON")))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

account_address   = w3.to_checksum_address(os.environ.get("WALLET_ADDRESS"))  
private_key       = os.environ.get("KEY_PRIVATE")

account_address

In [ ]:
ONE_INCH = '0x1111111254fb6c44bAC0beD2854e76F90643097d'
inch_router = w3.eth.contract(address=Web3.to_checksum_address(ONE_INCH), abi=abi_holder.inch_router)

In [ ]:
POOL_ADDRESS = '0x31083a78e11b18e450fd139f9abea98cd53181b7' 
pool = PoolV3(w3, POOL_ADDRESS)

amount_in = 1.
slippage = 0.1
min_out = (1.0 - slippage) * pool.get_amount_out(amount_in, zto=False)

In [ ]:
print(f"""
{pool.token0.symbol} / {pool.token1.symbol} [{pool.address}]
- fee: {pool.fee/(10**4)}
- current price: {pool.get_price(zto=False)}
- slippage: {slippage}

---

Your balance {pool.token1.get_balance(account_address)[1]}
{amount_in} {pool.token1.symbol} -> [{min_out} - {pool.get_amount_out(amount_in, zto=False)}] {pool.token0.symbol}
""")

In [ ]:
approve_trx = pool.token1.get_approve_tx(
    account_address, 
    ONE_INCH, 
    pool.token1.int_by_value(amount_in)
)

signed_approve_tx = w3.eth.account.sign_transaction(approve_trx, private_key)
tx_hash           = w3.eth.send_raw_transaction(signed_approve_tx.rawTransaction)

print(f"https://polygonscan.com/tx/{w3.to_hex(tx_hash)}")

In [ ]:
input("Подождать исполнения апрува")

In [ ]:
tx_swap_params = {
    'chainId':  w3.eth.chain_id,  
    'gas':      200000,
    'gasPrice': w3.to_wei(get_gas_price(), 'gwei'),
    'nonce':    w3.eth.get_transaction_count(account_address),
}

swap_tx = inch_router.functions.uniswapV3Swap(
    pool.token1.int_by_value(amount_in), 
    pool.token0.int_by_value(min_out),
    [pool.encode_address(zto=False)]
).build_transaction(tx_swap_params)

signed_usdt_tx = w3.eth.account.sign_transaction(swap_tx, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_usdt_tx.rawTransaction)

print(f"https://polygonscan.com/tx/{w3.to_hex(tx_hash)}")